# InceptionV3

Model to classify the food 101 images, based on the Inception V3
<br>
Tested and working with tensorflow 2.1.0 & keras 2.3.1 
- 67988 training images
- 22716 test images


In [1]:
# # Install Keras version 2.3.1
#keras, tf and restart
# !pip install keras==2.3.1

In [2]:
# pip install tensorflow==2.2.0

In [3]:
# !pip install bert-for-tf2

In [4]:
# %tensorflow_version 2.x

# Import all necessary libraries
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import tensorflow_hub as hub
import os
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential, Model 
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, MaxPooling1D, GlobalAveragePooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, Input 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau 
from keras.callbacks import TensorBoard
from keras import backend as K 
from keras.optimizers import Adam, SGD, Adadelta
from keras.regularizers import l2, l1
import cv2
from keras.callbacks import CSVLogger
from tensorflow.keras import callbacks
import sys
import time
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt

from keras.applications.inception_v3 import InceptionV3

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Using TensorFlow backend.


In [5]:
# Downloading zip file containing images

In [6]:
#Unzipping into "images" folder
# !unzip /content/drive/MyDrive/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs/standardized_datasets.zip

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd /content/drive/MyDrive/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs
%pwd

/content/drive/.shortcut-targets-by-id/14M403BjGCu2i1Ehk4R93S9AYX5y6oVIw/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs


'/content/drive/.shortcut-targets-by-id/14M403BjGCu2i1Ehk4R93S9AYX5y6oVIw/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs'

In [9]:
# Loading images from Google Drive
# train_data_dir = '/content/images/train'
# validation_data_dir = '/content/images/test'
train_data_dir = '/content/drive/MyDrive/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs/train'
validation_data_dir = '/content/drive/MyDrive/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs/val'
test_data_dir = '/content/drive/MyDrive/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs/test'
# nb_train_samples = 67988 
# nb_validation_samples = 22716
n_classes = 47 #101
epochs = 10
batch_size = 75

In [10]:
# Checking image format: if RGB channel is coming first or last so, model will check first and then input shape will be feeded accordingly.
img_width = 299
img_height = 299

if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

**Model**
<br>
- InceptionV3 base model, with imagenet weights loaded
- Average Pooling layer with pool size 8x8
- Dropout with probability 0.4
- Flatten layer
- Dense Layer of 101 neurons

In [11]:
#Model that enable the freezing of the resnet layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)

predictions = Dense(n_classes,
                    kernel_regularizer=l2(0.005),
                    activity_regularizer=l1(0.005), 
                    activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

<ipython-input-11-c4f805fe05a2>:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  model = Model(input=base_model.input, output=predictions)


In [12]:
#from tensorflow.keras.utils import plot_model
#plot_model(model, to_file='model_cnn.png')

In [13]:
# Compile the model using Stochastic Gradiend Descent (SGD) optimizer
model.compile(
    optimizer=SGD(lr=.01, momentum=.9), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [14]:
# Plot the model and save to file
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [15]:
# Train & Test Data Generators with image augmentation 

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True, 
    validation_split=0.2, # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    fill_mode='reflect')



test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    subset='training',
    class_mode='categorical')

# validation_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     seed = 11,
#     subset='validation',
#     class_mode='categorical')
validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

Found 526 images belonging to 47 classes.
Found 92 images belonging to 47 classes.


In [16]:
from keras.callbacks import ModelCheckpoint

In [17]:
# Setup callbacks and logs 
checkpoint_path = "InceptionV3/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
cp = ModelCheckpoint(checkpoint_path, monitor='val_accuracy',save_best_only=True,verbose=1, mode='max')
csv_logger = callbacks.CSVLogger('InceptionV3/InceptionV3.log')

In [18]:
# Reduce LR if no improvement on the test accuracy is observed
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=2, min_lr=0.00001)

In [19]:
# Loading best model weights
# model.load_weights('/content/drive/MyDrive/Image-and-Text-fusion-for-UPMC-Food-101-using-BERT-and-CNNs/InceptionV3/weights-improvement-10-0.55.hdf5')

In [20]:
len(train_generator)

8

**Model Training**

In [21]:
# model.fit(train_generator,
#           steps_per_epoch = 825 // batch_size,
#           epochs=10,
#           verbose=1,
#           callbacks=[csv_logger, reduce_lr]
#           )

In [22]:
batch_size = 32

In [ ]:
model.fit(train_generator,
          steps_per_epoch = 526 // batch_size,
          validation_data=validation_generator,
          #validation_split = 0.2,
          validation_steps= 92 // batch_size,
          epochs=10,
          verbose=1,
          callbacks=[cp, csv_logger]
          )

Epoch 1/10
16/16 [==============================] - 1116s 70s/step - loss: 4.3084 - accuracy: 0.0865 - val_loss: 4.7792 - val_accuracy: 0.0217

Epoch 00001: val_accuracy improved from -inf to 0.02174, saving model to InceptionV3/weights-improvement-01-0.02.hdf5
Epoch 2/10
16/16 [==============================] - 1075s 67s/step - loss: 2.5755 - accuracy: 0.6312 - val_loss: 4.8160 - val_accuracy: 0.0978

Epoch 00002: val_accuracy improved from 0.02174 to 0.09783, saving model to InceptionV3/weights-improvement-02-0.10.hdf5
Epoch 3/10
16/16 [==============================] - 1059s 66s/step - loss: 2.0896 - accuracy: 0.8222 - val_loss: 5.6664 - val_accuracy: 0.0543

Epoch 00003: val_accuracy did not improve from 0.09783
Epoch 4/10
16/16 [==============================] - 1069s 67s/step - loss: 1.8898 - accuracy: 0.9183 - val_loss: 6.7108 - val_accuracy: 0.1304

Epoch 00004: val_accuracy improved from 0.09783 to 0.13043, saving model to InceptionV3/weights-improvement-04-0.13.hdf5
Epoch 5/1

In [ ]:
#Fitting
model.fit(train_generator,
          steps_per_epoch = 526 // batch_size,
          validation_data=validation_generator,
          #validation_split = 0.2,
          validation_steps= 92 // batch_size,
          epochs=10,
          verbose=1,
          callbacks=[cp, csv_logger]
          )

In [ ]:
# Read log file
import pandas as pd
df = pd.read_csv('InceptionV3/InceptionV3.log')

In [ ]:
# Training and Test accuracy
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['epoch'], y=df['accuracy'],
                    mode='lines',
                    name='training'))

fig.add_trace(go.Scatter(x=df['epoch'], y=df['val_accuracy'],
                    mode='lines',
                    name='test'))


fig.update_layout(
    font_size = 20,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')

In [ ]:
# Training and Test loss
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['epoch'], y=df['loss'],
                    mode='lines',
                    name='training loss'))

fig.add_trace(go.Scatter(x=df['epoch'], y=df['val_loss'],
                    mode='lines',
                    name='test loss'))


fig.update_layout(
    font_size = 20,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')

**Model evaluation**

In [ ]:
test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

In [ ]:
batch_size
    

In [ ]:
#Evaluation
score = model.evaluate_generator(
    test_generator, 5
    )

print('Accuracy',score[1])